from tensorflow.python.eager import context
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.framework import tensor_util
from tensorflow.python.keras import activations
from tensorflow.python.keras import initializers
from tensorflow.python.layers import base as base_layer
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import partitioned_variables
from tensorflow.python.ops import random_ops
from tensorflow.python.ops import tensor_array_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import variables as tf_variables
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import nest
from tensorflow.python.util.deprecation import deprecated
from tensorflow.python.util.tf_export import tf_export

In [547]:
import numpy as np
import tensorflow as tf

x = np.array([[1, 1]])
c = 1.0 * np.asarray([[0, 1]])
h = 1.0 * np.asarray([[0, 1]])

In [548]:
x.shape

(1, 2)

In [549]:
c.shape

(1, 2)

In [550]:
h.shape

(1, 2)

In [551]:
c.size

2

In [552]:
def sigmoid_array(x):
     return 1 / (1 + np.exp(-x))
    
class lstmcell:
    def __init__(self, num, x_, c_, h_):
        self.x = x_ #input
        self.c = c_# 초기 state
        self.h = h_# 초기 output
        self.num = num

    
    def build(self):
        num_units = self.num #자료 갯수

        args = np.concatenate((self.x, self.h), axis=1)

        out_size = 4 * num_units
        proj_size = args.shape[-1]

        weights = np.ones([proj_size, out_size]) * 0.5

        out = np.matmul(args, weights)

        bias = np.ones([out_size]) * 0.5

        concat = out + bias

        i, j, f, o = np.split(concat, 4, 1)

        g = np.tanh(j)#Cell 현재값
        
        forget_bias = 1.0

        sigmoid_f = sigmoid_array(f + forget_bias)#forget gate

        sigmoid_array(i) * g

        new_c = c * sigmoid_f + sigmoid_array(i) * g

        new_h = np.tanh(new_c) * sigmoid_array(o)

        new_c = np.split(new_c, 1, 1)
        new_h = np.split(new_h, 1, 1)

        return new_h, new_c,

In [553]:
T = max_time = x.size
B = batch_size = len(x) #배치사이즈
D = input_depth = len(c)#디멘션
num_units = c.size #= 출력의 크기
#제로스테이트 = 배치*디멘션

In [554]:
max_time = x.size
batch_size = len(x) #배치사이즈
input_depth = len(c)
num_units = c.size #디멘션

In [555]:
cell = lstmcell(num_units, x, c, h)
output, state = cell.build()

In [556]:
def propagation(num_units, data, new_state, new_output):
    cell = lstmcell(num_units, x, new_state, new_output)
    output, state = cell.build()
    new_output = np.asarray(output)
    new_state = np.asarray(state)
    new_output = new_output.reshape(1, num_units)
    new_state = new_state.reshape(1, num_units)
    return new_state, new_output

In [646]:
new_state, new_output = c, h
states = np.array([0, 0])
predict = np.array([0.88490995, 1.84762319])
for i in range(10):
    new_state, new_output = propagation(num_units, x, new_state, new_output)
    global states
    states = states + (predict**2 - new_state**2)
    cost = states/(i+1)
    print(cost)
    print(new_output, new_state, "\n")

[[0.06207328 0.16763612]]
[[0.60828342 0.83409758]] [[0.84911268 1.8016868 ]] 

[[0.03435606 0.0927643 ]]
[[0.63780624 0.85798651]] [[0.88115083 1.8427748 ]] 

[[0.02313913 0.06247637]]
[[0.64097197 0.86057769]] [[0.88451136 1.84710881]] 

[[0.01737307 0.04690772]]
[[0.64130844 0.86085343]] [[0.88486764 1.84756859]] 

[[0.01390004 0.03753046]]
[[0.64134416 0.86088271]] [[0.88490546 1.8476174 ]] 

[[0.01158351 0.03127576]]
[[0.64134796 0.86088582]] [[0.88490948 1.84762258]] 

[[0.00992873 0.02680782]]
[[0.64134836 0.86088615]] [[0.8849099  1.84762313]] 

[[0.00868764 0.02345685]]
[[0.6413484  0.86088619]] [[0.88490995 1.84762319]] 

[[0.00772235 0.02085053]]
[[0.64134841 0.86088619]] [[0.88490995 1.84762319]] 

[[0.00695011 0.01876548]]
[[0.64134841 0.86088619]] [[0.88490995 1.84762319]] 



In [601]:
propagation(num_units, x, new_state, new_output)

(array([[0.88490995, 1.84762319]]), array([[0.64134841, 0.86088619]]))

In [602]:
new_state

array([[0.88490995, 1.84762319]])

In [603]:
new_output

array([[0.64134841, 0.86088619]])

In [604]:
states

array([[ 8.8090972 , 19.42487167]])

##########

X = tf.placeholder(shape=[T, B, D], dtype=tf.float32)
output_ta = tf.TensorArray(size=T, dtype=tf.float32)
input_ta = tf.TensorArray(size=T, dtype=tf.float32)
input_ta = input_ta.unstack(x)

def body(time, output_ta_t, state):
    xt = input_ta.read(time)
    new_output, new_state = lstmcell(num_units, xt, state, h).build()
    output_ta_t = output_ta_t.write(time, new_output)
    return (time+1, output_ta_t, new_state)

def condition(time, output, state):
    return time < T

time = 0
state = tf.zeros([B, D], dtype=tf.float32),

time_final, output_ta_final, state_final = tf.while_loop(
        cond=condition,
        body=body,
        loop_vars=(time, output_ta, state))

output_final = output_ta_final.stack()

#################

In [75]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())